In [15]:
#Put the review you want to be poetified here:
review = "I loved Applied Fundamentals of Machine Learning. The professors were great and the TAs too. \
It gave us the opportunity to learn how to use neural networks to solve practical problems"


In [5]:
from google.colab import drive
drive.mount('/content/drive')
home_directory = '/content/drive/MyDrive/Aps360 Project/'

Mounted at /content/drive


In [6]:
!pip install transformers
!pip install anvil-uplink
!pip install datasets
!pip install git+https://github.com/huggingface/transformers
!pip install rouge_score

     |████████████████████████████████| 2.6 MB 8.6 MB/s 
     |████████████████████████████████| 3.3 MB 56.1 MB/s 
     |████████████████████████████████| 895 kB 65.7 MB/s 
     |████████████████████████████████| 636 kB 72.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 62 kB 894 kB/s 
     |████████████████████████████████| 51 kB 205 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45230 sha256=07c95a75631e996a498e77af9d94afb10606dbf08b2959b57cf3a8cb4280eb7b
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py


     |████████████████████████████████| 264 kB 8.2 MB/s 
     |████████████████████████████████| 243 kB 62.7 MB/s 
     |████████████████████████████████| 118 kB 69.7 MB/s 
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-tehh8to2
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-tehh8to2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.10.0.dev0-py3-none-any.whl size=2644547 sha256=d8beda0de0f935e68960580072e1ea980614d023471b1e67803f639f1ad5a317
  Stored in directory: /tmp/pip-ephem-wheel-cache-harjk3wt/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.9.2
    Uninstalling transformers-4.9.2:
      Successfully uninstalled transformers-4.9.2


In [7]:
import numpy as np
import pandas as pd 
import csv

import random
import time
import datetime
import gc

import anvil.server
from anvil.google.drive import app_files


import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, pipeline

In [8]:
RANDOM_SEED = 73
MAX_LEN = 600

In [9]:
#Load the GPT2 tokenizer that will be used by PoemDataset to encode the poems. Add the 
#BOS,EOS and PAD tokens to the tokenized dictionary so that when we put these 
#Tokens around our poems to separate them, the tokenizer will know what to do with them.
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'sep_token':'<SEP>', 'pad_token': '<PAD>'}
tokenizer.add_special_tokens(special_tokens_dict)

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

4

In [10]:
torch.cuda.manual_seed_all(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device('cuda')

gc.collect()
torch.cuda.empty_cache() 

#If you are only evaluating the model and don't want to train it again,
#set training_desired to false. Otherwise set it to true and choose the 
#epoch you left off at last time to start Training from (0 if you haven't).
previous_state_dict_location = "model_amazon_review_poem_FINAL_part2"

#Setup the pretrained GPT2 model
configuration = GPT2Config(vocab_size=len(tokenizer), n_positions=MAX_LEN).from_pretrained('gpt2', output_hidden_states=True)
poem_model = GPT2LMHeadModel.from_pretrained('gpt2', config=configuration)
poem_model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Embedding(50261, 768)

In [11]:
poem_model.load_state_dict(torch.load("{}/{}".format(home_directory,previous_state_dict_location)))

poem_model.cuda()
poem_model = poem_model.to(device)

In [12]:
poem_model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50261, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [13]:
def generate_poem(prompt):
  #first, write the prompt to a test file
  with open('test.csv', 'w', newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_ALL)
    writer.writerow(['summary','text'])
    writer.writerow(['unknown',prompt])
  #then create the generated title of the review
  !python "/content/drive/MyDrive/Aps360 Project/reviewSummarizer/run_summarization.py" \
  --do_predict \
  --test_file "/content/test.csv" \
  --model_name_or_path "t5-small" \
  --do_train \
  --train_file "/content/drive/MyDrive/Aps360 Project/Datasets/amazonReviews/train/train_100.csv" \
  --source_prefix "summarize: " \
  --output_dir "/content/drive/MyDrive/Aps360 Project/reviewSummarizer" \
  --overwrite_output_dir \
  --per_device_train_batch_size=20 \
  --per_device_eval_batch_size=20 \
  --predict_with_generate \
  --text_column text \
  --summary_column summary \
  --max_train_samples 1 \
  --resume_from_checkpoint "/content/drive/MyDrive/Aps360 Project/reviewSummarizer/checkpoint-5000-safe"

  #load the generated title:
  file1 = open('/content/drive/MyDrive/Aps360 Project/reviewSummarizer/generated_predictions.txt', 'r',encoding="utf-8")
  lines = file1.readlines()
  prompt_summary = lines[0]

  original_summary = prompt_summary           
  prompt_summary = prompt_summary + "<SEP>"

  encoded = torch.tensor(tokenizer.encode(prompt_summary)).unsqueeze(0)
  generated = encoded.to(device)
  sample_outputs = poem_model.generate(
                                  generated, 
                                  do_sample=True,   
                                  top_k=50, 
                                  max_length=MAX_LEN,
                                  top_p=0.95, 
                                  num_return_sequences=1
                                  )
  return original_summary.upper() + '\n' + tokenizer.decode(sample_outputs[0], skip_special_tokens=True)[len(original_summary):]
  

In [18]:
poetic_review = generate_poem(review)


2021-08-13 21:00:30.175770: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
08/13/2021 21:00:31 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/13/2021 21:00:31 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
l

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [19]:
print(poetic_review)

GREAT APPLIED FUNDAMENTALS OF MACHINE LEARNING
The term “Machine Learning” was coined by Michael Krzanikoff and the University of Michigan Press in honor of Lawrence Nove—“Machine learning” is an applied mathematical method that combines the power of mathematical expression with the mathematical properties of applied mathematical process.

Before my years in applied mathematics I was the leader of the student body that developed the foundations of machine learning. The result is a comprehensive set of tools that are not only highly accurate and practical but also extremely flexible in applications.

I was happy to hear the latest developments in machine learning. Today I have another project in mind: to create a supercomputer that simmers and optimally reproduces the human brain to reduce our daily and daily suffering.

What kind of machine is this? I asked.

A super computer is a computer that works in many different ways—

—but every day a new computer is being built
by the likes of 